<a href="https://colab.research.google.com/github/Santosh-Gupta/AbstractNet/blob/master/cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
import os
import sys
import pandas as pd
import numpy as np
import gc
from urllib import request
import json
import itertools
import gzip
import shutil
import ast
import pickle 

# Cleaning


In [0]:
def dlandopen(url, file, json):
    request.urlretrieve(url, file )
    with gzip.open(file, 'rb') as f_in:
        with open(json, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(file)
    f = open(json)
    mylistH = f.readlines()
    del f
    return mylistH

In [0]:
def firstDataframe(mylist, start, finish):
    data = []

    for line in itertools.islice(mylist , start , finish):
       data.append(json.loads(line))
    df = pd.DataFrame(data)
    del data
    gc.collect()
    return df

In [0]:
def processData(dftbr, type):
  #TODO: Should only keep files with inCitations or outCitations + of type specified. Pruning would come next.
  dftbr['totalCitations'] = dftbr['inCitations'] + dftbr['outCitations']
  dftbr.drop(['journalVolume','journalPages','year','authors','sources','doiUrl', 'inCitations', 'outCitations'], axis = 1)
  dftbr =  dftbr[( dftbr.astype(str)['totalCitations'] != '[]' )] #remove any paper with no citations
  dftbr = dftbr[dftbr.fieldsOfStudy.apply(lambda x: type[0] in x or type[1] in x)]
  return dftbr

In [0]:
#Corpus goes from 000 to 180
url_template = "https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2020-04-10/s2-corpus-{:03}.gz"
file_template = 'json{:03}.gz'
json_template = 'json{:03}'
save_template = "/content/gdrive/My Drive/TF2.2Data/CleanedData/df{:03}.json"

In [0]:
#To Finish
i = 140
while i <= 140:
  first_list = dlandopen(url_template.format(i), file_template.format(i), json_template.format(i))
  print("File Unzipped: " + str(i))
  raw_df = firstDataframe(first_list, 0, len(first_list))
  print("Raw Dataset Created: " + str(i))
  first_clean_df = processData(raw_df, ["Biology", "Medicine"])
  del raw_df
  print("First Clean: " + str(i))
  print(first_clean_df.shape)
  first_clean_df.to_json(save_template.format(i))
  del first_clean_df
  print("Saved: " + str(i))
  i += 1

File Unzipped: 140
Raw Dataset Created: 140
First Clean: 140
(157996, 21)
Saved: 140


# Pruning

In [0]:
df_template = "/content/gdrive/My Drive/TF2.2Data/CleanedData/df{:03}.json"
paper_set = set()
i = 0
while i <= 184:
  print(i)
  df = pd.read_json(df_template.format(i))
  b = set(df['id'])
  paper_set.update(b)
  i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184


In [0]:
len(paper_set)

29144681

In [0]:
  df = pd.read_json(df_template.format(100))
  df

,entities,journalVolume,journalPages,pmid,fieldsOfStudy,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue
4,[],2,85 - 92,21863141,"[Computer Science, Medicine]",2011,"[98a7371c4f39a796404092d38fd76fc2550003ef, 755...",https://semanticscholar.org/paper/f539e1a6cc59...,,f539e1a6cc5987c438a583349ee31f3901821646,"[{'name': 'Nora Ibargoyen-Roteta', 'ids': ['14...",,INTRODUCTION\nA system for monitoring the use ...,[24e9f9c8f3a284b969c08682bfe3079277b20e61],[http://journal-cdn.frontiersin.org/article/11...,Post-Marketing Health Technology Monitoring. T...,10.3389/fphar.2011.00045,[Medline],https://doi.org/10.3389/fphar.2011.00045,Front. Pharmacol.
37,[],abs/1908.06601,,,[Computer Science],2019,[3e74cc19f919747f6398cb93762cdec5da672212],https://semanticscholar.org/paper/9f2762d65daf...,,9f2762d65dafdb68b6c7701de6002c0b9d02889f,"[{'name': 'Mike Ji', 'ids': ['30435998']}]",ArXiv,The most important notations of Communicating ...,[],"[https://arxiv.org/pdf/1908.06601v2.pdf, http:...",Implicit Recursive Characteristics of STOP,,[DBLP],,ArXiv
104,[],,1-5,,[Computer Science],2018,"[6cd592bd2fb1ebdccd8f416afacb2caf354725d3, ede...",https://semanticscholar.org/paper/10be86574eb6...,,10be86574eb62d7ab855b72f4efdbfef42c1a4bc,"[{'name': 'Nasim Alamdari', 'ids': ['145199506...",2018 IEEE Signal Processing in Medicine and Bi...,This paper presents the development of a perso...,[a6a2f4ebe38d2488d8ad95f90588c1b9acf9a268],[https://www.ieeespmb.org/2018/papers/l01_03.pdf],A Real-Time Personalized Noise Reduction Smart...,10.1109/spmb.2018.8615620,[],https://doi.org/10.1109/spmb.2018.8615620,2018 IEEE Signal Processing in Medicine and Bi...
136,[],,,,[Computer Science],2018,[],https://semanticscholar.org/paper/d83b616a9a15...,,d83b616a9a15e21e098f8eee3a9fb28a28e31cce,"[{'name': 'Setareh Yazdkhasti', 'ids': ['27403...",,The optimal performance of the conventional Ka...,"[3d1873d29c392ccaaa51f378720a185b48f18361, 919...",[],Multi Sensor Fusion Based on Adaptive Kalman F...,10.1007/978-3-319-65283-2_17,[],https://doi.org/10.1007/978-3-319-65283-2_17,
149,[],,413-418,,[Computer Science],2008,"[7a39814beeff2175e0dd72b87d1492e2c04b084e, 337...",https://semanticscholar.org/paper/9ad0c372e282...,,9ad0c372e2822727d65b095644989a814416fc8c,"[{'name': 'A.K. Kaviani', 'ids': ['49303051']}...",2008 11th International Conference on Optimiza...,A hybrid wind/photovoltaic/fuel cell generatio...,[],[],Optimal design of a reliable hydrogen-based st...,10.1109/OPTIM.2008.4602442,[],https://doi.org/10.1109/OPTIM.2008.4602442,2008 11th International Conference on Optimiza...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999548,[],,1780-1783,,[Computer Science],2018,"[29c808b346526fbb6027e67942b62a40a549f019, eb4...",https://semanticscholar.org/paper/21d32ce8f5d7...,,21d32ce8f5d7c5aa10ecfce0c2247933eda2a7f6,"[{'name': 'Yuan Shen', 'ids': ['1916963']}, {'...",IGARSS 2018 - 2018 IEEE International Geoscien...,The deep learning has been widely used in scen...,"[b55a05426d854de0e84215d02f953560d0492d4d, db4...",[https://doi.org/10.1109/IGARSS.2018.8518209],Csrs-Siat: A Benchmark Remote Sensing Dataset ...,10.1109/IGARSS.2018.8518209,[DBLP],https://doi.org/10.1109/IGARSS.2018.8518209,IGARSS 2018 - 2018 IEEE International Geoscien...
999581,[],,5499-5502,17281498,"[Computer Science, Medicine]",2005,"[164cab9b61d66ed252b311171cb66a5bfeff8f45, 361...",https://semanticscholar.org/paper/5c29c61824c0...,,5c29c61824c0fa9f725de608ba9caafa47a87774,"[{'name': 'Guo Qing Yang', 'ids': ['2816473']}...",2005 IEEE Engineering in Medicine and Biology ...,Dynamic computed tomography (CT) imaging aims ...,"[2e1ae89740e69bfe9ecd627e2ab985bf40837643, 554...",[],A 3D Static Heart Model From a MSCT Data Set,10.1109/IEMBS.2005.1615728,[Medline],https://doi.org/10.1109/IEMBS.2005.1615728,2005 IEEE Engineering in Medicine and Biology ...
999588,[],,29-32 vol.1,,[Computer Science],1989,"[9344052be5

In [0]:
pruned_file_template = "/content/gdrive/My Drive/TF2.2Data/PrunedData/prdf{:03}.json"
i = 0
while i <= 184:
  print(i)
  df = pd.read_json(df_template.format(i))
  print(df.shape)
  df2 = df[df["totalCitations"].transform(lambda x: bool(set(x)&paper_set))]
  print(df2.shape)
  df2.to_json(pruned_file_template.format(i))
  i += 1

0
(157966, 21)
(150609, 21)
1
(157486, 21)
(150116, 21)
2
(158061, 21)
(150746, 21)
3
(157459, 21)
(150117, 21)
4
(157556, 21)
(150234, 21)
5
(157953, 21)
(150566, 21)
6
(158156, 21)
(150934, 21)
7
(157691, 21)
(150451, 21)
8
(158176, 21)
(150772, 21)
9
(156987, 21)
(149842, 21)
10
(157472, 21)
(150166, 21)
11
(157658, 21)
(150414, 21)
12
(157035, 21)
(149724, 21)
13
(157864, 21)
(150666, 21)
14
(158064, 21)
(150627, 21)
15
(157183, 21)
(150050, 21)
16
(157147, 21)
(149882, 21)
17
(157427, 21)
(150140, 21)
18
(157540, 21)
(150276, 21)
19
(157583, 21)
(150255, 21)
20
(157465, 21)
(150099, 21)
21
(157588, 21)
(150333, 21)
22
(157291, 21)
(150143, 21)
23
(158027, 21)
(150645, 21)
24
(158098, 21)
(150803, 21)
25
(157389, 21)
(150234, 21)
26
(157433, 21)
(150180, 21)
27
(157727, 21)
(150535, 21)
28
(157029, 21)
(149836, 21)
29
(158372, 21)
(151041, 21)
30
(157891, 21)
(150649, 21)
31
(157118, 21)
(149811, 21)
32
(157589, 21)
(150279, 21)
33
(157628, 21)
(150300, 21)
34
(157158, 21)
(149884,

In [0]:
df2

,entities,journalVolume,journalPages,pmid,fieldsOfStudy,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,totalCitations
3,[],1704,59-67,30244112,"[Biology, Medicine]",2019,[],https://semanticscholar.org/paper/8130b5d814f2...,,8130b5d814f29958a7310febc3ef63f05e6b4532,"[{'name': 'Li Liu', 'ids': ['49480323']}, {'na...",Brain Research,The medial prefrontal cortex (mPFC) receives d...,"[fed312bd88ebf8847d709e7966ab4e4a11f7ce8a, 44e...",[https://api.elsevier.com/content/article/pii/...,Dopamine D1 receptor in the medial prefrontal ...,10.1016/j.brainres.2018.09.024,[Medline],https://doi.org/10.1016/j.brainres.2018.09.024,Brain Research,"[fed312bd88ebf8847d709e7966ab4e4a11f7ce8a, 44e..."
4,[],68,92 - 93,18863685,"[Medicine, Biology]",1948,[],https://semanticscholar.org/paper/e8ec41d5d290...,,e8ec41d5d29093c973ef34a4a0d6f2aef8ba26c2,"[{'name': 'James H. Leathem', 'ids': ['3693292...",Proceedings of the Society for Experimental Bi...,Summary The increase in seminal vesicle and ve...,[b0fe5322769854f9f4e9a84fe5f4a2a19b82e53e],[],Volume of Oil and Route of Administration as F...,10.3181/00379727-68-16401,[Medline],https://doi.org/10.3181/00379727-68-16401,Proceedings of the Society for Experimental Bi...,[b0fe5322769854f9f4e9a84fe5f4a2a19b82e53e]
5,[],,,,[Biology],1991,[],https://semanticscholar.org/paper/a5c7e3a7832a...,,a5c7e3a7832a9f952f169e75a0bd5e4be17dffd4,"[{'name': 'Enrique Ortega', 'ids': ['145186606...",,,"[941f48029f859c07d45182f5cec8c41fe3215cfd, 785...",[],Possible interactions between the Fcε receptor...,10.1093/intimm/3.4.333,[],https://doi.org/10.1093/intimm%2F3.4.333,,"[941f48029f859c07d45182f5cec8c41fe3215cfd, 785..."
7,[],,,,[Medicine],2005,[],https://semanticscholar.org/paper/25df9ca76020...,,25df9ca76020bcb4293bdf916c7f778ea7f0b267,"[{'name': 'Vincent Dousset', 'ids': ['3351983'...",,Resume Les cephalees de tension constituent un...,[45bd0a1dcf88f246c8915eeba0337a875edbb287],[],Céphalées de tension,10.1016/j.emcn.2005.01.005,[],https://doi.org/10.1016/j.emcn.2005.01.005,,[45bd0a1dcf88f246c8915eeba0337a875edbb287]
11,[],,,,[Biology],2010,[],https://semanticscholar.org/paper/3c6ba105a275...,,3c6ba105a27564431c0446ec1e0632c312ce966d,"[{'name': 'Shin-Ru Shih', 'ids': ['40575409']}...",,Background: Influenza viruses are a major caus...,"[3f1e8c2463d2bf5f62a09a515936d6b90bce4e9e, 443...",[],Researchcompound BPR1P0034 with potent and sel...,,[],,,"[3f1e8c2463d2bf5f62a09a515936d6b90bce4e9e, 443..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995207,[],,,,[Biology],2015,[],https://semanticscholar.org/paper/809049888eb0...,,809049888eb08517fdd20a225a3a276377be1964,"[{'name': 'John Murphy', 'ids': ['50064503']},...",,1. Detrimental impacts of excessive fine-grain...,"[753973b7f2e0a04eb92a6aafcf79b735776aa0be, aa4...",[],Development of a biotic index using stream mac...,10.1111/fwb.12627,[],https://doi.org/10.1111/fwb.12627,,"[753973b7f2e0a04eb92a6aafcf79b735776aa0be, aa4..."
995210,[],16,322 - 328,24155762,[Medicine],2013,"[887beab98c4f5da3989197372b8900a966c37bc7, e4a...",https://semanticscholar.org/paper/d2842c17d22e...,,d2842c17d22e5832fecc655b8fcd71ea1ceb9d06,"[{'name': 'Chang Suk Park', 'ids': ['2286557']...",,PURPOSE\nWe aimed to determine the sensitivity...,"[24ff36dd6cf88a0738e638b21cd1233bed99fab4, ac2...",[http://ftp.ncbi.nlm.nih.gov/pub/pmc/ce/fc/jbc...,Detection of Breast Cancer in Asymptomatic and...,10.4048/jbc.2013.16.3.322,[Medline],https://doi.org/10.4048/jbc.2013.16.3.322,Journal of breast cancer,"[24ff36dd6cf88a0738e638b21cd1233bed99fab4, ac2..."
995212,[],,,,[Medicine],1985,[],https://semanticscholar.org/paper/2071200121ba...,,2071200121bafc01df32f9c7ec06655fa213f3d9,"[{'name': 'Eugene H. Courtiss', 'ids': ['68523...",,Only for you today! Discover your favourite te...,"[6cb84d75530b7cbc569db7762fcb29e3e7a2225c, 8b5...",[http://palico.co.uk/techniques/in/techniques_...,Techniques in Aesthetic R

In [0]:
df2

,entities,journalVolume,journalPages,pmid,fieldsOfStudy,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue
62,[],,685-688,,[Computer Science],2007,[06b6b105e94a8987b464a7cb28a2fecd060de864],https://semanticscholar.org/paper/4d44dafcaa63...,,4d44dafcaa637a230afd8efaaabb8891621cd42c,"[{'name': 'Hisashi Handa', 'ids': ['3251294']}]",2007 IEEE International Conference on Networki...,"Recently, Multitask learning, which can cope w...","[b48395464261992a9934b904dd3fd692fdb93847, 0b1...",[https://doi.org/10.1109/ICNSC.2007.372862],Evolutionary Computation on Multitask Reinforc...,10.1109/ICNSC.2007.372862,[DBLP],https://doi.org/10.1109/ICNSC.2007.372862,2007 IEEE International Conference on Networki...
85,[],4,2975-2980 vol.4,,[Computer Science],2001,"[e0bd6e0c494ca277aeaa5ef00da41da5b2600b9c, 8c9...",https://semanticscholar.org/paper/ed1dff67126f...,,ed1dff67126f0218f5031c156227e96568f29134,"[{'name': 'Lan Yi Wang', 'ids': ['48169620']},...",IJCNN'01. International Joint Conference on Ne...,"In our previous work (2000), we proposed a nov...",[],[],Towards better capturing inter-speaker informa...,10.1109/IJCNN.2001.938851,[],https://doi.org/10.1109/IJCNN.2001.938851,IJCNN'01. International Joint Conference on Ne...
142,[],,,,[Computer Science],1989,[],https://semanticscholar.org/paper/2cf395d88248...,,2cf395d882486b280d60bd4b732a3d60ca54b564,"[{'name': 'Iztok Tvrdy', 'ids': ['2130829']}]",,Abstract The paper describes modelling of a di...,"[b3ca4d14b118302488edda0619512e2814dfb962, 614...",[],Formal modelling of telematic services using L...,10.1016/0165-6074(89)90214-7,[],https://doi.org/10.1016/0165-6074%2889%2990214-7,
185,[],,2158-2161,29060324,"[Computer Science, Medicine]",2017,"[e1fabb876bfb5e6522c8101aba2ab30497acc65f, 0bc...",https://semanticscholar.org/paper/f13f03aa0557...,,f13f03aa0557e02fa70d75a460931a2dd375be73,"[{'name': 'Rosti Lemdiasov', 'ids': ['27022531...",2017 39th Annual International Conference of t...,A new design approach for the design of transm...,"[987a5f77f286013897c5fd309daffaa9e2fc9386, a22...",[https://www.wikidata.org/entity/Q50070760],Transmit coil design for Wireless Power Transf...,10.1109/EMBC.2017.8037282,"[Medline, DBLP]",https://doi.org/10.1109/EMBC.2017.8037282,2017 39th Annual International Conference of t...
198,[],,,,[Computer Science],2013,"[e836971e392022331d4ec1e467097a187eaa838e, 4be...",https://semanticscholar.org/paper/20a44372040f...,,20a44372040f71cde42b1f46c5e9a36deaa9f428,"[{'name': 'Er. Gurpreet Kaur', 'ids': ['701182...",,"Day by day, the deployments of the networks ar...",[],[https://www.ijert.org/research/implementation...,Implementation of Quality of Services (QoS) fo...,,[],,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999430,[],,895-906,,[Computer Science],2014,"[4ced3fd73ccc79ba7f72544e6ba9b1f87127150b, c40...",https://semanticscholar.org/paper/b372f40bafc4...,,b372f40bafc4a4a4c4dbedb7634e468a5028c97e,"[{'name': 'Marc Gamell', 'ids': ['2765481']}, ...",SC14: International Conference for High Perfor...,Application resilience is a key challenge that...,"[18d5384b9d63e32190e691be990d1def366cf882, 030...",[http://doi.ieeecomputersociety.org/10.1109/SC...,"Exploring Automatic, Online Failure Recovery f...",10.1109/SC.2014.78,[DBLP],https://doi.org/10.1109/SC.2014.78,SC14: International Conference for High Perfor...
999479,[],,,,[Computer Science],2019,"[e8c83300990e0336d6cf2ef5ff4c75fd998a6771, 273...",https://semanticscholar.org/paper/4cba4ae121a3...,,4cba4ae121a3addc5938e00d0416849d1e5b286f,"[{'name': 'Bahram Samadi', 'ids': ['1453597446...",,The suspension footbridges are very flexible d...,[],[http://www.ije.ir/article_95712_195a5a91b7c27...,Dynamic Analysis of Suspension Footbridges Usi...,10.5829/ije.2019.32.10a.07,[],https://doi.org/10.5829/ije.2019.32.10a.07,
999541,[],,1-7,,[Computer Science],2008,"[5dfbd031a9d8d46ae24b4ea19ce1cbd25fefad9e, 70d...",https://sem

In [0]:
df_template = "/content/gdrive/My Drive/NLRData/PrunedData/prdf{:03}.json"
paper_set = set()
i = 0
while i <= 184:
  print(i)
  df = pd.read_json(df_template.format(i))
  b = set(df['id'])
  paper_set.update(b)
  i+=1
print(len(paper_set))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
6277085
